In [1]:
# !pip install scikeras

# Homework 2

For this assignment, you will be developing an artificial neural network to classify data given in the __[Dry Beans Data Set](https://archive.ics.uci.edu/ml/datasets/Dry+Bean+Dataset#)__. This data set was obtained as a part of a research study by Selcuk University, Turkey, in which a computer vision system was developed to distinguish seven different registered varieties of dry beans with similar features. More details on the study can be found in the following __[research paper](https://www.sciencedirect.com/science/article/pii/S0168169919311573)__. <br>
### **Make sure to use the lecture notebook on an introduction to keras and cross validation located [here](https://colab.research.google.com/drive/1ksEGL7SJ_wutCIyPYx7Loe5EPdOij6dJ?usp=sharing) and [here](https://colab.research.google.com/drive/1C9Mwf1J2ril1Q4l6n2BjQMb8YaFySG5_?usp=sharing)**.

## About the Data Set
Seven different types of dry beans were used in a study in Selcuk University, Turkey, taking into account the features such as form, shape, type, and structure by the market situation. A computer vision system was developed to distinguish seven different registered varieties of dry beans with similar features in order to obtain uniform seed classification. For the classification model, images of 13611 grains of 7 different registered dry beans were taken with a high-resolution camera. Bean images obtained by computer vision system were subjected to segmentation and feature extraction stages, and a total of 16 features - 12 dimensions and 4 shape forms - were obtained from the grains.

Number of Instances (records in the data set): __13611__

Number of Attributes (fields within each record, including the class): __17__

### Data Set Attribute Information:

1. __Area (A)__ : The area of a bean zone and the number of pixels within its boundaries.
2. __Perimeter (P)__ : Bean circumference is defined as the length of its border.
3. __Major axis length (L)__ : The distance between the ends of the longest line that can be drawn from a bean.
4. __Minor axis length (l)__ : The longest line that can be drawn from the bean while standing perpendicular to the main axis.
5. __Aspect ratio (K)__ : Defines the relationship between L and l.
6. __Eccentricity (Ec)__ : Eccentricity of the ellipse having the same moments as the region.
7. __Convex area (C)__ : Number of pixels in the smallest convex polygon that can contain the area of a bean seed.
8. __Equivalent diameter (Ed)__ : The diameter of a circle having the same area as a bean seed area.
9. __Extent (Ex)__ : The ratio of the pixels in the bounding box to the bean area.
10. __Solidity (S)__ : Also known as convexity. The ratio of the pixels in the convex shell to those found in beans.
11. __Roundness (R)__ : Calculated with the following formula: (4piA)/(P^2)
12. __Compactness (CO)__ : Measures the roundness of an object: Ed/L
13. __ShapeFactor1 (SF1)__
14. __ShapeFactor2 (SF2)__
15. __ShapeFactor3 (SF3)__
16. __ShapeFactor4 (SF4)__

17. __Classes : *Seker, Barbunya, Bombay, Cali, Dermosan, Horoz, Sira*__

### Libraries that can be used :
- NumPy, SciPy, Pandas, Sci-Kit Learn, TensorFlow, Keras
- Any other library used during the lectures and discussion sessions.

### Other Notes
- Don't worry about not being able to achieve high accuracy, it is neither the goal nor the grading standard of this assignment.
- Discussion and Lecture materials should be helpful for doing the assignments.
- The homework submission should be a .ipynb file.


In [2]:
!git clone https://github.com/ucsd-cse151a-w24/hw2.git

fatal: destination path 'hw2' already exists and is not an empty directory.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, train_test_split


/tmp/ipykernel_674613/88789289.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd



## Exercise 1 : Building a Feed-Forward Neural Network(50 points)

### Exercise 1.1 : Data Preprocessing (10 points)

- As the classes are categorical, use one-hot encoding to represent the set of classes. You will find this useful when developing the output layer of the neural network.
- Normalize each field of the input data using the min-max normalization technique.



In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("/home/jovyan/hw2/hw2/Dry_Beans_Dataset.csv")

df_class = df.Class.values
df_input = df.drop("Class", axis='columns')

encoder = OneHotEncoder()
df_class = encoder.fit_transform(df_class.reshape(-1,1)).toarray()

scalar = MinMaxScaler()
df_input = scalar.fit_transform(df_input)

print(df_class)
print(df_input)

[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
[[0.10422894 0.19869385 0.24696724 ... 0.25625262 0.25607398 0.95775147]
 [0.05574818 0.11439553 0.13549858 ... 0.44060883 0.40014396 0.94114256]
 [0.02833951 0.05573059 0.0789897  ... 0.55951279 0.45268999 0.953829  ]
 ...
 [0.11890041 0.20032671 0.21806318 ... 0.3683878  0.44586352 0.94973786]
 [0.09760628 0.21793961 0.27496949 ... 0.18508428 0.13632834 0.91152138]
 [0.11287981 0.1947353  0.21430398 ... 0.36344336 0.42736307 0.93107481]]


### Exercise 1.2 : Training and Testing the Neural Network (40 points)

Design a 4-layer (4 hidden layers and this does not include the input or output layer) artificial deep neural network, specifically a feed-forward multi-layer perceptron (using the sigmoid activation function), to classify the type of 'Dry Bean' given the other attributes in the data set, similar to the one mentioned in the paper above. Please note that this is a **multi-class classification** problem so select the right number of nodes accordingly for the input and output layers.

For training and testing the model, split the data into training and testing set by __90:10__ and use the training set for training the model and the test set to evaluate the model performance.

Consider the following hyperparameters while developing your model :

- Model type: Keras Sequential
- Make sure your input layer matches the size of your X matrix
- Number and type of hidden layers: 4 and Dense
- Number of nodes in each hidden layer: 12
- Learning rate should be 0.3
- Number of epochs should be 100
- The sigmoid function is to be used as the activation function in each layer
- Your output layer has to use a sigmoid function and the number of outputs should match the shape of your y
- Stochastic Gradient Descent should be used to minimize the error rate

__Requirements once the model has been trained :__

- A confusion matrix for all classes, specifying the true positive, true negative, false positive, and false negative cases for each category in the class
- Since we do have OHE output (multi-class output) you will need to either reshape or argmax your outputs. Make sure they have already been thresholded as well i.e. look at yhat and do you see 1's and 0's?
- The accuracy and mean squared error (MSE) of the model
- The precision and recall for each label in the class

__Notes :__

- Splitting of the dataset should be done __after__ the data preprocessing step.
- The mean squared error (MSE) values obtained __should be positive__.


In [5]:
from sklearn.model_selection import train_test_split
from scikeras.wrappers import KerasClassifier
from sklearn.metrics import confusion_matrix, mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from sklearn.metrics import confusion_matrix, accuracy_score

2024-02-12 06:52:45.507534: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-12 06:52:45.545665: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-12 06:52:45.545722: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-12 06:52:45.546554: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-12 06:52:45.553585: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-12 06:52:45.554580: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [6]:
x_train, x_test, y_train, y_test = train_test_split(df_input, df_class, test_size=0.1, random_state = 100)

def dnnmodel():
    model = Sequential([
        Dense(12, activation='sigmoid', input_dim=x_train.shape[1]),
        Dense(12, activation='sigmoid'),
        Dense(12, activation='sigmoid'),
        Dense(12, activation='sigmoid'),
        Dense(y_train.shape[1], activation='sigmoid')
    ])
    model.compile(optimizer=SGD(learning_rate=0.3), loss='categorical_crossentropy', metrics=['accuracy'])
    return(model)

classifier = dnnmodel()

history = classifier.fit(x_train, y_train, epochs = 100, verbose = 0)

loss, accuracy = classifier.evaluate(x_test, y_test)
y_pred = classifier.predict(x_test)

y_test = y_test.argmax(axis=1)
y_pred = y_pred.argmax(axis=1)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
cm = conf_matrix
fp = cm.sum(axis=0) - np.diag(cm)
fn = cm.sum(axis=1) - np.diag(cm)
tp = np.diag(cm)
tn = cm.sum() - (fp + fn + tp)

accuracy = accuracy_score(y_test, y_pred)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
mse = mean_squared_error(y_test, y_pred)

2024-02-12 06:52:47.417563: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


43/43 [==============================] - 0s 518us/step


In [7]:
print(cm)
i=0
for category in encoder.categories_[0]:
    print(category)
    print(f"tp: {tp[i]}, fp: {fp[i]}, tn: {tn[i]}, fn: {fn[i]}")
    print("Precision:", precision[i])
    print("Recall:", recall[i])
    i+=1
    print()

print("y_pred:", y_pred)
print("Accuracy:", accuracy)
print("MSE:", mse)

[[118   0   6   0   0   1   6]
 [  0  58   0   0   0   0   0]
 [  4   0 151   0   5   0   1]
 [  0   0   0 350   1   4   2]
 [  0   0   2   2 169   0   2]
 [  2   0   0   4   0 197   3]
 [  0   0   0  47  10   5 212]]
BARBUNYA
tp: 118, fp: 6, tn: 1225, fn: 13
Precision: 0.9516129032258065
Recall: 0.9007633587786259

BOMBAY
tp: 58, fp: 0, tn: 1304, fn: 0
Precision: 1.0
Recall: 1.0

CALI
tp: 151, fp: 8, tn: 1193, fn: 10
Precision: 0.949685534591195
Recall: 0.937888198757764

DERMASON
tp: 350, fp: 53, tn: 952, fn: 7
Precision: 0.8684863523573201
Recall: 0.9803921568627451

HOROZ
tp: 169, fp: 16, tn: 1171, fn: 6
Precision: 0.9135135135135135
Recall: 0.9657142857142857

SEKER
tp: 197, fp: 10, tn: 1146, fn: 9
Precision: 0.9516908212560387
Recall: 0.9563106796116505

SIRA
tp: 212, fp: 14, tn: 1074, fn: 62
Precision: 0.9380530973451328
Recall: 0.7737226277372263

y_pred: [5 5 4 ... 6 5 3]
Accuracy: 0.92143906020558
MSE: 0.6659324522760646


## Exercise 2 : k-fold Cross Validation (20 points)

In order to avoid **using biased models**, use 10-fold cross validation to generalize the model from Ex1.2 on the given data set. You can choose a n_repeats value of 1-5

__Requirements :__
- Print the accuracy values during each iteration of the **cross validation** not the iterations per epoch or the epochs
- Print the overall average accuracy per each n_fold value, look at the documentation for the scoring parameter

__Note :__ This is a classification problem not a regresson problem. Make sure to choose the correct Keras library.

In [8]:
from sklearn.model_selection import RepeatedKFold, cross_val_score

estimator = KerasClassifier(model=dnnmodel, epochs=100, batch_size=100, verbose=0)
kfold = RepeatedKFold(n_splits = 10, n_repeats = 1)
results = cross_val_score(estimator, x_train, y_train, cv=kfold, n_jobs = 1, scoring = "accuracy")  # 1 cpus



In [9]:
i=1
for result in results:
  print(f"{i}: {result}")
  i+=1

print("Average accuracy:",results.mean())

1: 0.9036734693877551
2: 0.9069387755102041
3: 0.9110204081632653
4: 0.8677551020408163
5: 0.8710204081632653
6: 0.8963265306122449
7: 0.9110204081632653
8: 0.88
9: 0.8906122448979592
10: 0.9076797385620915
Average accuracy: 0.8946047085500867


## Exercise 3 : Hyperparameter Tuning (25 points)

Use either grid search or random search methodology to find the optimal number of nodes required in each hidden layer, as well as the optimal learning rate and the different activation functions or optimization approaches, [keras_tuner examples](https://keras.io/guides/keras_tuner/getting_started/) such that the accuracy of the model is maximum for the given data set.

__Requirements :__
- The set of optimal hyperparameters
- The maximum accuracy achieved using this set of optimal hyperparameters

__Note :__ Hyperparameter tuning takes a lot of time to execute. Make sure that you choose the appropriate number of each hyperparameter (preferably 3 of each), and that you allocate enough time to execute your code.

In [10]:
# !pip install keras_tuner

In [11]:
import keras_tuner

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state = 100)

def dnnmodel(hp):
    model = Sequential([
        Dense(units=hp.Int("units1", min_value=6, max_value=18, step=6),
              activation=hp.Choice("activation1", ["sigmoid", "relu", "tanh"]),
              input_dim=x_train.shape[1]),
        Dense(units=hp.Int("units2", min_value=6, max_value=18, step=6),
              activation=hp.Choice("activation2", ["sigmoid", "relu", "tanh"])),
        Dense(units=hp.Int("units3", min_value=6, max_value=18, step=6),
              activation=hp.Choice("activation3", ["sigmoid", "relu", "tanh"])),
        Dense(units=hp.Int("units4", min_value=6, max_value=18, step=6),
              activation=hp.Choice("activation4", ["sigmoid", "relu", "tanh"])),
        Dense(y_train.shape[1], activation=hp.Choice("activation5", ["sigmoid", "relu", "tanh"]))
    ])
    learning_rate = hp.Float("lr", min_value=0.2, max_value=0.3, step=0.1)
    model.compile(optimizer=SGD(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return(model)

hp = keras_tuner.HyperParameters()

tuner = keras_tuner.GridSearch(
    hypermodel=dnnmodel,
    objective="val_accuracy",
    max_trials=10,
    seed=15,
    executions_per_trial=5,
    #hyperparameters=None,
    tune_new_entries=True,
    allow_new_entries=True,
    #max_retries_per_trial=0,
    max_consecutive_failed_trials=3
)

tuner.search(x_train, y_train, epochs=100, validation_data=(x_val, y_val))

tuner.search_space_summary()

tuner.results_summary()

models = tuner.get_best_models(num_models=1)
print(models[0].summary())

Reloading Tuner from ./untitled_project/tuner0.json
Search space summary
Default search space size: 10
units1 (Int)
{'default': None, 'conditions': [], 'min_value': 6, 'max_value': 18, 'step': 6, 'sampling': 'linear'}
activation1 (Choice)
{'default': 'sigmoid', 'conditions': [], 'values': ['sigmoid', 'relu', 'tanh'], 'ordered': False}
units2 (Int)
{'default': None, 'conditions': [], 'min_value': 6, 'max_value': 18, 'step': 6, 'sampling': 'linear'}
activation2 (Choice)
{'default': 'sigmoid', 'conditions': [], 'values': ['sigmoid', 'relu', 'tanh'], 'ordered': False}
units3 (Int)
{'default': None, 'conditions': [], 'min_value': 6, 'max_value': 18, 'step': 6, 'sampling': 'linear'}
activation3 (Choice)
{'default': 'sigmoid', 'conditions': [], 'values': ['sigmoid', 'relu', 'tanh'], 'ordered': False}
units4 (Int)
{'default': None, 'conditions': [], 'min_value': 6, 'max_value': 18, 'step': 6, 'sampling': 'linear'}
activation4 (Choice)
{'default': 'sigmoid', 'conditions': [], 'values': ['sigmoi

In [12]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 06 summary
Hyperparameters:
units1: 6
activation1: sigmoid
units2: 6
activation2: sigmoid
units3: 6
activation3: sigmoid
units4: 6
activation4: tanh
activation5: sigmoid
lr: 0.2
Score: 0.919294822216034

Trial 00 summary
Hyperparameters:
units1: 6
activation1: sigmoid
units2: 6
activation2: sigmoid
units3: 6
activation3: sigmoid
units4: 6
activation4: sigmoid
activation5: sigmoid
lr: 0.2
Score: 0.913222336769104

Trial 09 summary
Hyperparameters:
units1: 6
activation1: sigmoid
units2: 6
activation2: sigmoid
units3: 6
activation3: sigmoid
units4: 12
activation4: sigmoid
activation5: sigmoid
lr: 0.2
Score: 0.9065621733665467

Trial 03 summary
Hyperparameters:
units1: 6
activation1: sigmoid
units2: 6
activation2: sigmoid
units3: 6
activation3: sigmoid
units4: 6
activation4: relu
activation5: sigmoid
lr: 0.2
Score: 0.7860920608043671

Trial 01 summary
Hyperparameters:

In [21]:
models = tuner.get_best_models(num_models=1)
models[0].summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 102       
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dense_2 (Dense)             (None, 6)                 42        
                                                                 
 dense_3 (Dense)             (None, 6)                 42        
                                                                 
 dense_4 (Dense)             (None, 7)                 49        
                                                                 
Total params: 277 (1.08 KB)
Trainable params: 277 (1.08 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Exercise 4 - Collaborative Statement (5 points)

It is mandatory to include a Statement of Collaboration in each submission, that follows the guidelines below.
Include the names of everyone involved in the discussions (especially in-person ones), and what was discussed.
All students are required to follow the academic honesty guidelines posted on the course website. For
programming assignments in particular, I encourage students to organize (perhaps using Piazza) to discuss the
task descriptions, requirements, possible bugs in the support code, and the relevant technical content before they
start working on it. However, you should not discuss the specific solutions, and as a guiding principle, you are
not allowed to take anything written or drawn away from these discussions (no photographs of the blackboard,
written notes, referring to Piazza, etc.). Especially after you have started working on the assignment, try to restrict
the discussion to Piazza as much as possible, so that there is no doubt as to the extent of your collaboration.

In [13]:
# I use some of the course material(notebook) provide by Prof.
# I use the link provided in HW.
# Link: https://keras.io/guides/keras_tuner/getting_started/
# I use the document Numpy, Pandas